In [2]:
import cv2
import numpy as np
import pandas as pd
import pickle as cPickle
from matplotlib import pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,confusion_matrix,multilabel_confusion_matrix,recall_score
from sklearn.preprocessing import normalize
from sklearn import svm
from datetime import datetime

In [3]:
train_images_filenames = cPickle.load(open('resources/train_images_filenames.dat','rb'))
test_images_filenames = cPickle.load(open('resources/test_images_filenames.dat','rb'))
train_labels = cPickle.load(open('resources/train_labels.dat','rb'))
test_labels = cPickle.load(open('resources/test_labels.dat','rb'))

In [4]:
SIFTdetector = cv2.SIFT_create(nfeatures=300)

In [5]:
def compute_dense_sift(gray,sift,step):
    step_size = step
    kp = [cv2.KeyPoint(x, y, step_size) for y in range(0, gray.shape[0], step_size) 
                                        for x in range(0, gray.shape[1], step_size)]
    dense_feat = sift.compute(gray, kp)
    dense_feat_des = dense_feat[1]
    return dense_feat_des

In [ ]:
step=5

today = datetime.now()
dt_string = today.strftime("%H:%M:%S")
print(f"{dt_string} started doing step={step}")

Train_descriptors = []
Train_label_per_descriptor = []

for filename,labels in zip(train_images_filenames,train_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
#     kpt,des=SIFTdetector.detectAndCompute(gray,None)
    des=compute_dense_sift(gray,SIFTdetector, step)
    des=normalize(des)
    Train_descriptors.append(des)
    Train_label_per_descriptor.append(labels)

D=np.vstack(Train_descriptors)

k=128
codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
codebook.fit(D)

visual_words=np.zeros((len(Train_descriptors),k),dtype=np.float32)
for i in range(len(Train_descriptors)):
    words=codebook.predict(Train_descriptors[i])
    visual_words[i,:]=np.bincount(words,minlength=k)

# knn = KNeighborsClassifier(n_neighbors=37,n_jobs=-1,metric='manhattan')
# knn.fit(visual_words, train_labels) 


12:51:54 started doing step=5


In [7]:
visual_words

array([[ 5., 24.,  3., ...,  3., 14., 48.],
       [ 2.,  4.,  0., ..., 12.,  0., 11.],
       [ 5., 74., 15., ...,  1.,  0., 18.],
       ...,
       [ 3.,  0., 10., ...,  1.,  1.,  8.],
       [ 0.,  0.,  1., ...,  0.,  0.,  1.],
       [47., 24., 17., ...,  1.,  6.,  0.]], dtype=float32)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
visual_words=scaler.fit_transform(visual_words)

In [9]:
lin_clf = svm.LinearSVC(max_iter=2000)
lin_clf.fit(visual_words, train_labels)

# rbf_svc = svm.SVC(kernel='rbf')
# rbf_svc.fit(visual_words, train_labels)

visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
for i in range(len(test_images_filenames)):
    filename=test_images_filenames[i]
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
#     kpt,des=SIFTdetector.detectAndCompute(gray,None)
    des=compute_dense_sift(gray,SIFTdetector, step)
    des=normalize(des)
    words=codebook.predict(des)
    visual_words_test[i,:]=np.bincount(words,minlength=k)

scaler = StandardScaler()
visual_words_test=scaler.fit_transform(visual_words_test)
scores = cross_val_score(lin_clf, visual_words_test, test_labels, cv=5)
# scores = cross_val_score(rbf_svc, visual_words_test, test_labels, cv=5)
accuracy=scores.mean()*100

print(accuracy)

/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("L

66.67203435319378


/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [10]:
# lin_clf = svm.LinearSVC(max_iter=2000)
# lin_clf.fit(visual_words, train_labels)

rbf_svc = svm.SVC(kernel='rbf')
rbf_svc.fit(visual_words, train_labels)

visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
for i in range(len(test_images_filenames)):
    filename=test_images_filenames[i]
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
#     kpt,des=SIFTdetector.detectAndCompute(gray,None)
    des=compute_dense_sift(gray,SIFTdetector, step)
    des=normalize(des)
    words=codebook.predict(des)
    visual_words_test[i,:]=np.bincount(words,minlength=k)

scaler = StandardScaler()
visual_words_test=scaler.fit_transform(visual_words_test)
# scores = cross_val_score(lin_clf, visual_words_test, test_labels, cv=5)
scores = cross_val_score(rbf_svc, visual_words_test, test_labels, cv=5)
accuracy=scores.mean()*100

print(accuracy)

78.19108964036499


In [1]:
def get_pyramid_image_cells(image,level):
    # CURRENTLY WORKING ONLY FOR LEVELS 0,1,2 (1,4, and 16 cells)
    M = (image.shape[0])//(level+1)
    N = (image.shape[1])//(level+1)
    tiles = [image[x:x+M,y:y+N] for x in range(0,image.shape[0],M) for y in range(0,image.shape[1],N)]
    #print(image.shape)
    #print("level=", level)
    #print("returning " + str(len(tiles)) + " tiles")
    return tiles


'''
for filename,labels in zip(train_images_filenames,train_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    break

tiles = get_pyramid_image_cells(gray,2)
'''

'\nfor filename,labels in zip(train_images_filenames,train_labels):\n    ima=cv2.imread(filename)\n    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)\n    break\n\ntiles = get_pyramid_image_cells(gray,2)\n'

## Calculating a histogram per cell and concatenating them
Compute a histogram for each part of the image and put these histograms together in a weighted concatenation 
(since there will be less words overall in a smaller cell).
The weighted concatenation can be a simple normalization of every histogram so the area of the histogram adds up to 1.
A pyramid of l levels will yield in 2^(2l) histograms n_words = k*(2**(2*pyramid_levels)).
the final image histogram is a concatenated vector  of  the  bin  values  of  all  the  histograms in  the  pyramid.
concatenation method used in:
http://lear.imag.fr/pub/203-bosch-civr07.pdf
https://core.ac.uk/download/pdf/82695491.pdf

TODO: Initialisation of concatenated_histogram. 

In [24]:
step=5

today = datetime.now()
dt_string = today.strftime("%H:%M:%S")
print(f"{dt_string} started doing step={step}")

Train_descriptors = []
Train_label_per_descriptor = []
pyramid_levels = 1 # level zero has 1 tile, level 1 has 4 tiles => 5 histograms that will be concatenated
for filename,labels in zip(train_images_filenames,train_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    # break the image into pieces
    image_cells = []
    for i in range(pyramid_levels+1):

        level_cells = get_pyramid_image_cells(gray,level=i)
        image_cells = image_cells + level_cells
        # compute descriptors for each tile
    Train_descriptors_cell = []
    for cell in image_cells:
        des=compute_dense_sift(cell,SIFTdetector, step)
        des=normalize(des)
        Train_descriptors_cell.append(des)
        
        Train_descriptors.append(Train_descriptors_cell)
        Train_label_per_descriptor.append(labels)

12:48:44 started doing step=5
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1


(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

...

(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
level= 0
returning 1 tiles
(256, 256)
level= 1
returning 4 tiles
(256, 256)
l

In [ ]:
# train all cells as separate images
D=np.vstack([des for descriptors_cells in Train_descriptors for des in descriptors_cells])

k=128
codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
codebook.fit(D)

In [22]:
print(k*(2**(2*pyramid_levels)))
visual_words=np.zeros((len(Train_descriptors),k*(2**(2*pyramid_levels))),dtype=np.float32)
for idx,cells in enumerate(Train_descriptors):

    image_histograms = []
    for image_cell in enumerate(cells):
        cell_words = codebook.predict(image_cell)
        image_histograms.append(np.bincount(cell_words,minlength=k)) 
        # concatenated histogram has k*(2**(2*pyramid_levels)) bins

    concatenated_histogram=image_histograms[0].copy()
    for cell_idx in range(1,len(image_histograms)):
        # concatenate histograms
        concatenated_histogram = np.concatenate((concatenated_histogram,image_histograms[cell_idx]))
    visual_words[idx,:]=concatenated_histogram
    break
    
    #visual_words[idx,:]=np.bincount(words,minlength=k*(2**(2*pyramid_levels))) # this will just be longer as levels 
                                                                                # increase 
                                                                                # see: k*(2**(2*pyramid_levels)) long

# knn = KNeighborsClassifier(n_neighbors=37,n_jobs=-1,metric='manhattan')
# knn.fit(visual_words, train_labels) 

512


ValueError: setting an array element with a sequence.

In [11]:
def (gray, levels=0, )

SyntaxError: invalid syntax (<ipython-input-11-250528976da2>, line 1)

In [92]:
visual_words[0,:].shape

(128,)